In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk import Address, TokenPayment, ProxyNetworkProvider
from multiversx_sdk.core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder
from multiversx_sdk.core.transaction_builders import DefaultTransactionBuildersConfiguration
from utils.contract_data_fetchers import SimpleLockEnergyContractDataFetcher
from utils.contract_retrievers import retrieve_simple_lock_energy_by_address
from utils.utils_chain import Account, decode_merged_attributes
from utils import decoding_structures
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(chain_id=env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

In [ ]:
contract_address = env.ENERGY_CONTRACT

In [ ]:
from multiversx_sdk import AccountNonceHolder, UserSigner
from pathlib import Path

user = env.USER1
user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))
owner = Account(pem_file=env.USER1_PEM)
owner.address = env.OWNER

energy contract upgrade & init

In [ ]:
energy_contract = retrieve_simple_lock_energy_by_address(contract_address.bech32())
data_fetcher = SimpleLockEnergyContractDataFetcher(contract_address, proxy.url)

In [ ]:
from utils.contract_retrievers import retrieve_locked_asset_factory_by_address
locked_asset = retrieve_locked_asset_factory_by_address("erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th")

In [ ]:
from utils.contract_retrievers import retrieve_proxy_by_address
proxy_contract = retrieve_proxy_by_address("erd1qqqqqqqqqqqqqpgqrc4pg2xarca9z34njcxeur622qmfjp8w2jps89fxnl")
proxy_contract.version = 1

In [ ]:
owner.sync_nonce(proxy)
energy_contract.contract_upgrade(owner, proxy, Path.home()+"/projects/dex/dex-v2/sc-dex-rs/locked-asset/energy-factory/output/energy-factory.wasm", [env.LKMEX, "erd1qqqqqqqqqqqqqpgqjpt0qqgsrdhp2xqygpjtfrpwf76f9nvg2jpsg4q7th", 0, [], []])

lkmex/xmex whitelistings

In [ ]:
owner.sync_nonce(proxy)
energy_contract.set_transfer_role_locked_token(owner, proxy, [env.FORWARDER_CONTRACT.bech32()])

In [ ]:
owner.sync_nonce(proxy)
locked_asset.set_transfer_role_for_contract(owner, proxy, env.FORWARDER_CONTRACT.bech32())

In [ ]:
owner.sync_nonce(proxy)
proxy_contract.set_transfer_role_locked_lp_token(owner, proxy, env.FORWARDER_CONTRACT.bech32())

In [ ]:
owner.sync_nonce(proxy)
proxy_contract.set_transfer_role_locked_farm_token(owner, proxy, env.FORWARDER_CONTRACT.bech32())

In [ ]:
owner.sync_nonce(proxy)
energy_contract.resume(owner, proxy)

migrateOldTokens

In [ ]:
from multiversx_sdk import TransactionsConverter

user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.meta_esdt_from_integer(env.LKMEX, int("3e4112", 16), 100000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="migrateOldTokens",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()

converter = TransactionsConverter()
print("Transaction:", converter.transaction_to_dictionary(tx))
print("Transaction data:", tx.data.decode())

In [ ]:
from multiversx_sdk import TransactionComputer

tx.nonce = user_nonce_holder.get_nonce_then_increment()

tx_computer = TransactionComputer()
tx.signature = user_signer.sign(tx_computer.compute_bytes_for_signing(tx))
proxy.send_transaction(tx)

In [ ]:
raw_energy_entry = data_fetcher.get_data('getEnergyEntryForUser', [env.USER3.serialize()])
energy_entry_user = decode_merged_attributes(raw_energy_entry, decoding_structures.ENERGY_ENTRY)
print(energy_entry_user)
raw_energy_entry = data_fetcher.get_data('getEnergyEntryForUser', [env.FORWARDER_CONTRACT.serialize()])
energy_entry_contract = decode_merged_attributes(raw_energy_entry, decoding_structures.ENERGY_ENTRY)
print(energy_entry_contract)

In [ ]:
raw_energy_entry = data_fetcher.get_data('getEnergyEntryForUser', [env.USER3.serialize()])
after_energy_entry_user = decode_merged_attributes(raw_energy_entry, decoding_structures.ENERGY_ENTRY)
print(after_energy_entry_user)
raw_energy_entry = data_fetcher.get_data('getEnergyEntryForUser', [env.FORWARDER_CONTRACT.serialize()])
after_energy_entry_contract = decode_merged_attributes(raw_energy_entry, decoding_structures.ENERGY_ENTRY)
print(after_energy_entry_contract)

In [ ]:
diff = after_energy_entry_user['amount'] - energy_entry_user['amount']
tokens = after_energy_entry_user['total_locked_tokens'] - energy_entry_user['total_locked_tokens']
print(diff, tokens)
diff = after_energy_entry_contract['amount'] - energy_entry_contract['amount']
tokens = after_energy_entry_contract['total_locked_tokens'] - energy_entry_contract['total_locked_tokens']
print(diff, tokens)